Extract Tables from PDF and convert to Excel sheet with Paddle OCR text detection and recognition

#https://www.youtube.com/watch?v=HZh31OGiQRQ

In [66]:
%pip install pdf2image
%pip install poppler-utils
%pip install paddlepaddle
%pip install layoutparser
%pip install paddle
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached paddle-1.0.2.tar.gz (579 kB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/private/var/folders/m5/8mkz0vqn4v5dmsqp6w27mb880000gn/T/pip-install-_wvhyuxy/paddle_2578ac7c239a410abc9a6c42e2f6f26c/setup.py", line 3, in <module>
          import paddle
        File "/private/var/folders/m5/8mkz0vqn4v5dmsqp6w27mb880000gn/T/pip-install-_wvhyuxy/paddle_2578ac7c239a410abc9a6c42e2f6f26c/paddle/__init__.py", line 5, in <module>
    

# Convert pdf to image

In [67]:
from pdf2image import convert_from_path
images = convert_from_path('./report_only_first_table_type.pdf', poppler_path='/opt/homebrew/bin')

In [68]:
%mkdir pages

for i in range(len(images)):
    images[i].save('pages/page'+str(i)+'.jpg','JPEG')

mkdir: pages: File exists


In [69]:
#PublayNet

#PaddlePaddle/PaddleOCR Github, layout detection
#ppstructure/layout/ 

# Layout extraction

In [70]:
import cv2
import layoutparser as lp
import paddle

image = cv2.imread("./pages/page0.jpg")
image = image[..., ::-1]
device = paddle.set_device('cpu')

In [71]:


# load model
# model = lp.PaddleDetectionLayoutModel(config_path="lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config",
#                                 threshold=0.5,
#                                 label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
#                                 enforce_cpu=False,
#                                 enable_mkldnn=True)

# ORIGINAL config_path = "lp://PubLayNet/ppyolov2_r50vd_dcn_365e_publaynet/config"

model = lp.PaddleDetectionLayoutModel(config_path="lp://paddledetection/TableBank/ppyolov2_r50vd_dcn_365e",
                                label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"},
                                enforce_cpu=True) 


# def init_model():
#     model = lp.Detectron2LayoutModel(
#         os.path.join(os.getcwd(), 'code/ParseServer/models/PubLayNet/faster_rcnn_R_50_FPN_3x/config.yml'),
#         os.path.join(os.getcwd(),
#                      'code/ParseServer/models/PubLayNet/faster_rcnn_R_50_FPN_3x/model_final.pth'),
#         {0: "Text", 1: "Title", 2: "List", 3: "Table", 4: "Figure"})
#     return model


# model = init_model()


#detect
layout = model.detect(image)

In [72]:
from layoutparser import PaddleDetectionLayoutModel

In [73]:
def test_only_effdet_model():

    # When all the backeds are not installed, it should 
    # elicit only ImportErrors
    
    config = "lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config"
    model = Detectron2LayoutModel(config)
    image = cv2.imread("./pages/page0.jpg")
    layout = model.detect(image)
    
    with pytest.raises(ImportError):
        from layoutparser import EfficientDetLayoutModel
        from layoutparser import PaddleDetectionLayoutModel

In [74]:
# pip install -U layoutparser-0.0.1-py3-none-any.whl

In [75]:
# %pip install paddlepaddle-gpu
%pip install "paddleocr>=2.0.1"
%pip install protobuf==3.20.2
%git clone https://github.com/PaddlePaddle/PaddleOCR.git

Note: you may need to restart the kernel to use updated packages.
  Using cached protobuf-3.20.2-py2.py3-none-any.whl (162 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.23.4
    Uninstalling protobuf-4.23.4:
      Successfully uninstalled protobuf-4.23.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%git` not found.


In [76]:
# paddleOCR and drawocr
from paddleocr import PaddleOCR, draw_ocr

# extra utilities
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image
import webcolors
import os


In [86]:
def jpg_to_df(path):
    image_path = path
    ocr = PaddleOCR(use_angle_cls=True, lang='en')


    image_cv = cv2.imread(image_path)
    
    if image_cv is not None:
        test_result = ocr.ocr(image_path, cls=True)
        # Rest of your code to process the OCR results
        # ...
    else:
        print(f"Image not found or could not be read: {image_path}")
        return pd.DataFrame() 
    
    # boxes, texts, and probabilities from result
    res = test_result[0]
    boxes = [line[0] for line in res]
    texts = [line[1][0] for line in res]
    probabilities = [line[1][1] for line in res]
    
    image_boxes = image_cv.copy()
    image_height, image_width = image_cv.shape[0], image_cv.shape[1]
    for box, text in zip(boxes, texts):
        cv2.rectangle(image_boxes, (int(box[0][0]), int(box[0][1]), int(box[2][0]), int(box[2][1])), (0, 0, 255), 1)
        cv2.putText(image_boxes, text, (int(box[0][0]), int(box[0][1])), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (222, 0, 0), 1)

    # Returns the coordinates of the middle of the box
    def top_left(box):
        return box[0]


    ### RECONSTRUCTION ###

    im = image_cv.copy()

    hor_boxes = []
    vert_boxes = []

    for box in boxes:
        x_h, x_v = 0, int(box[0][0])
        y_h, y_v = int(box[0][1]), 0 

        width_h, width_v = image_width, int(box[2][0] - box[0][0])
        height_h, height_v = int(box[2][1] - box[0][1]), image_height

        hor_boxes.append([x_h, y_h, x_h + width_h, y_h + height_h])
        vert_boxes.append([x_v, y_v, x_v + width_v, y_v + height_v])

        cv2.rectangle(im, (x_h, y_h), (x_h + width_h, y_h + height_h), (255, 0, 0), 1)
        cv2.rectangle(im, (x_v, y_v), (x_v + width_v, y_v + height_v), (0, 255, 0), 1)

    cv2.imwrite('hor_vert.jpg', im)


    ### NON MAX SUPPRESSION ###

    hor_out = tf.image.non_max_suppression(
        hor_boxes,
        probabilities,
        max_output_size = 1000,
        iou_threshold = .1,
        score_threshold = float('-inf'), 
        name = None
    )

    hor_lines = np.sort(np.array(hor_out))

    im_nms = image_cv.copy()
    for val in hor_lines:
        cv2.rectangle(im_nms, (int(hor_boxes[val][0]), int(hor_boxes[val][1])), (int(hor_boxes[val][2]), int(hor_boxes[val][3])), (0, 0, 255), 1)
    cv2.imwrite('im_nms.jpg', im_nms)

    vert_out = tf.image.non_max_suppression(
        vert_boxes,
        probabilities,
        max_output_size = 1000,
        iou_threshold = .1,
        score_threshold = float('-inf'), 
        name = None
    )
    vert_lines = np.sort(np.array(vert_out))

    im_nms2 = im_nms.copy()

    for val in vert_lines:
        cv2.rectangle(im_nms2, (int(vert_boxes[val][0]), int(vert_boxes[val][1])), (int(vert_boxes[val][2]), int(vert_boxes[val][3])), (255, 0, 0), 1)
    cv2.imwrite('im_nms2.jpg', im_nms2) 


    ### CHANGE TO CSV ###

    out_array = [["" for i in range(len(vert_lines))] for j in range(len(hor_lines))]

    underordered_boxes = []
    for i in vert_lines:
        underordered_boxes.append(vert_boxes[i][0])
    ordered_boxes = np.array(np.argsort(underordered_boxes))

    # intersection over union
    def iou(box_1, box_2):

        x_1 = max(box_1[0], box_2[0])
        y_1 = max(box_1[1], box_2[1])
        x_2 = min(box_1[2], box_1[2])
        y_2 = min(box_1[3], box_2[3])

        inter = abs(max(x_2 - x_1, 0) * max(y_2 - y_1, 0))
        if inter == 0:
            return 0

        area_1 = abs((box_1[2] - box_1[0]) * (box_1[3] - box_1[1]))
        area_2 = abs((box_2[2] - box_2[0]) * (box_2[3] - box_2[1]))

        return inter / float(area_1 + area_2 - inter)

    # obtain the intersection of a given pair of vertical and horizontal boxes
    def intersection(box_1, box_2):
        return [box_2[0], box_1[1], box_2[2], box_1[3]]

    for i in range(len(hor_lines)):
        for j in range(len(vert_lines)):
            resultant = intersection(hor_boxes[hor_lines[i]], vert_boxes[vert_lines[ordered_boxes[j]]])
            for b in range(len(boxes)):
                the_box = [boxes[b][0][0], boxes[b][0][1], boxes[b][2][0], boxes[b][2][1]]
                if iou(resultant, the_box) > .1:
                    out_array[i][j] = texts[b]

    df = pd.DataFrame(out_array)

    # remove duplicates entries in rows and replace with 0
    # replace zero entries with NaN 
    mask = df.apply(pd.Series.duplicated, 1) & df.astype(bool)
    removed_df = df.mask(mask, 0)
    for col in removed_df.columns:
        removed_df[col].replace(to_replace = 0, value = pd.NA, inplace=True)
    removed_df.groupby([3], axis=0, as_index=False).sum()

    df.to_csv('sample.csv')
    removed_df.to_csv('fixed_sample.csv')

    ### GROUPING CORRECTION ###

    rgb_image = Image.open(image_path)
    pix = rgb_image.load()

    color_by_row = ['' for i in range(len(hor_lines))]

    hor_color_indices = [hor_boxes[line][1] for line in hor_lines]
    hor_color_indices

    for i in range(len(color_by_row)):
        color = pix[(50, hor_color_indices[i])]
        try:
            color_by_row[i] = webcolors.rgb_to_name(color)
        except ValueError:
            color_by_row[i] = 'gray'

    removed_df['color'] = color_by_row

    def group_rows(df):
        # Create an empty list to store rows of the new DataFrame
        new_rows = []

        current_color = None
        current_row = None

        for index, row in df.iterrows():
            if row['color'] == current_color:
                current_row = [str(current_row[i]) + '\n' + 
                               str(row.iloc[i]) if str(row.iloc[i]) != '<NA>' 
                               else current_row[i] for i in range(len(row))]
            else:
                # If the color of the current row is different or 'color' column does not exist,
                # create a new row in the new DataFrame and update the current color group
                if current_row is not None:
                    new_rows.append(current_row)
                current_row = row.tolist()
                current_color = row['color']

        # Append the last row to the new DataFrame
        if current_row is not None:
            new_rows.append(current_row)

        # Convert the list of rows into a new DataFrame
        new_df = pd.DataFrame(new_rows, columns=df.columns)
        return new_df

    # Assuming you have already defined the 'group_rows' function and provided the DataFrame 'removed_df' as an argument
    result_df = group_rows(removed_df)
    result_df = result_df.drop('color', axis=1)

    def clean_value(value):
        return str(value).replace('<NA>', '').strip()
    for index, row in result_df.iterrows():
        result_df.loc[index] = [clean_value(value) for value in row]

    return result_df

# Writing On All Pages of the Report

In [87]:
directory_path = './pages'
df_concat = pd.concat([jpg_to_df('./pages/' + path) for path in os.listdir(directory_path) if 
                       os.path.exists(directory_path) and os.path.isdir(directory_path)], ignore_index=True)

[2023/07/25 16:07:00] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/winstontang/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/winstontang/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_

In [90]:
df_copy = df_concat.copy(deep=True)

your_lab = False

for index, row in df_copy.iterrows():
    if not your_lab and 'YOURLAB' in str(row[0]):
        your_lab = True
    elif your_lab and 'YOURLAB' in str(row[0]):
        df_copy = df_copy.drop(index, axis=0)


df_copy.to_csv('test.csv')

In [80]:
# df_concat = pd.concat([df_concat.iloc[0], df_concat[df_concat[0].str.contains("YOURLAB") == False]])

In [81]:
# df_concat.to_csv('test.csv')

## Try Other Table Format

In [91]:
p_a_th = './test_pages2/0040.jpg'
jpg_to_df(p_a_th).to_csv('new_test.csv')

[2023/07/25 17:29:10] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/Users/winstontang/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/Users/winstontang/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_